# Qiskit Nature

Qiskit Nature is the application module dealing with problems in _natural sciences_.
Its goal is to provide end-user oriented modules which enable the fast solution of problems in _chemistry_, _physics_ and _biology_.

In general, any application is split into **Problems** and **Solvers** which are combined like so:
<img src="Nature_YT_Workflow.svg"/>
This modular approach permits the reusability of a solver for multiple problems.

Nature currently provides the following problems and solvers:

- **Problems**:
    - `ElectronicStructureProblem`
    - `VibrationalStructureProblem`
    - `ProteinFoldingProblem`
    
- **Solvers**:
    - `GroundStateEigensolver` (wraps Terra's `MinimumEigensolver`)
    - `AdaptVQE`
    - `ExcitedStatesEigensolver` (wraps Terra's `Eigensolver`)
    - `QEOM`
    - `BopesSampler`

Depending on your actual application, the internals of your problem can take different forms.
For example, the `ElectronicStructureProblem` (which we will be discussing today) looks like this:
<img src="Nature_YT_ElectronicStructureProblem.svg"/>

A similar modularity exists for the solvers. Generally these are provided by the `qiskit_nature.algorihtms` module which provide convenient wrappers of the actual algorithms implement in Qiskit Terra.
Here is an example of the `GroundStateEigensolver` which we will be looking at today:
<img src="Nature_YT_GroundStateEigensolver.svg"/>

## Electronic Structure

Let us now dive into the `ElectronicStructureProblem`. First, to cover some basics, let us talk about the general molecular Hamiltonian,

$$
\mathcal{H} = - \sum_I \frac{\nabla_{R_I}^2}{M_I} - \sum_i \frac{\nabla_{r_i}^2}{m_e} - \sum_I\sum_i  \frac{Z_I e^2}{|R_I-r_i|} + \sum_i \sum_{j>i} \frac{e^2}{|r_i-r_j|} + \sum_I\sum_{J>I} \frac{Z_I Z_J e^2}{|R_I-R_J|}
$$

From left to right these summands are:
- the nuclear kinetic energy
- the electronic kinetic energy
- the nuclei-electron attraction energy
- the electronic repulsion energy
- the nuclear repulsion energy

Since the nuclei are much heavier than the electrons they do not move on the same time scale and, thus, the behavior of nuclei and electrons can be decoupled. This is known as the _Born-Oppenheimer approximation_.

In practive this allows us to tackle the electronic problem first, with the nuclear coordinates entering merely as constant parameters. This gives us the _non-relativistic time-independent Schroedinger equation_:

$$
\mathcal{H}_{\text{el}} |\Psi_{n}\rangle = E_{n} |\Psi_{n}\rangle
$$

where 

$$
\mathcal{H}_{\text{el}} = - \sum_i \frac{\nabla_{r_i}^2}{m_e} - \sum_I\sum_i  \frac{Z_I e^2}{|R_I-r_i|} + \sum_i \sum_{j>i} \frac{e^2}{|r_i-r_j|}.
$$

In particular the ground state energy is given by:
$$
E_0 = \frac{\langle \Psi_0 | H_{\text{el}} | \Psi_0 \rangle}{\langle \Psi_0 | \Psi_0 \rangle}
$$
where $\Psi_0$ is the ground state of the system.

In order to solve this problem we must prepare the state $\Psi_0$ on a quantum computer.

A good starting point to do this, is to start from the solution based on the Hartree-Fock (HF) method which approxmates the N-body problem with $N$ 1-body problems.
This can be solved efficiently on a classical computer and results in the exact exchange energy of the electrons but lacks any correlation interactions.

Thus, with our quantum-computing algorithm we aim at correcting the HF-estimate by adding the missing correlation.

### Second Quantization

Up until now we have used the so-called _first quantization_ notation. However, in Qiskit we more commonly express our problems in the _second quantization_ form.
This notation allows us to write our Hamiltonian as a weighted sum of **creation** and **annihilation** operator products.

Thus, our Hamiltonian takes the form:

$$
\hat{H}_{elec}=\sum_{pq} h_{pq} \hat{a}^{\dagger}_p \hat{a}_q + 
\frac{1}{2} \sum_{pqrs} g_{pqrs}  \hat{a}^{\dagger}_p \hat{a}^{\dagger}_q \hat{a}_r  \hat{a}_s
$$
with the 1-body integrals
$$
h_{pq} = \int \phi^*_p(r) \left( -\frac{1}{2} \nabla^2 - \sum_{I} \frac{Z_I}{R_I- r} \right)   \phi_q(r)dr
$$
and 2-body integrals
$$
g_{pqrs} = \int \frac{\phi^*_p(r_1)  \phi^*_q(r_2) \phi_r(r_2)  \phi_s(r_1)}{|r_1-r_2|}dr_1dr_2.
$$

The beauty of the second quantization formalism is that not only operators but also **states** are written using the _creation_ and _annihilation_ operators.
This means that our system's wave function can be represented as an **occupation number vector**. This enables a very straight forward notation of _ground_ and _excited states_:

<img src="onv.png" width=800/>

### Molecular Orbitals

We sneakily introduced the concept of **molecular orbitals** which are obtained from the _atomic orbitals_ (given by our chosen basis) via a unitary transformation. In other words: each molecular orbital is a linear combination of all atomic orbitals.

<img src="water_mo_diagram.jpg"/>

Each MO can hold up to two electrons, one spin-up (_alpha_) and one spin-down (_beta_) electron.

In Qiskit we will need to map these two cases onto the quantum computer separately. Thus, we are working with **spin orbitals** which are identical to the MOs but restrict the spin in either the up or down configuration, depending on the nature of the orbital.

### The `ElectronicStructureProblem`

In [1]:
from qiskit_nature.drivers import Molecule
from qiskit_nature.drivers.second_quantization import ElectronicStructureMoleculeDriver, ElectronicStructureDriverType
from qiskit_nature.operators.second_quantization import FermionicOp
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
from qiskit_nature.transformers.second_quantization.electronic import ActiveSpaceTransformer

/home/oss/Files/Qiskit/.direnv/python-3.9.7/lib64/python3.9/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'


The `ElectronicStructureProblem` makes it easy for us to represent a problem of the form which we just discussed.
As an input, this kind of problem takes a so called `Driver` which is a concept in Qiskit Nature that allows us to couple to other (classical) computational chemistry codes which compute the 1- and 2-body integrals ($h_{pq}$ and $g_{pqrs}$) for us.

For this simple example, we will be using the generic `Molecule`-based interface and tell it to use `PySCF` as the classical computational backend.

In [2]:
molecule = Molecule(
    # coordinates are given in Angstrom
    geometry=[
        ["O", [0.0, 0.0, 0.115]],
        ["H", [0.0, 0.754, -0.459]],
        ["H", [0.0, -0.754, -0.459]],
    ],
    multiplicity=1,  # = 2*spin + 1
    charge=0,
)

In [3]:
driver = ElectronicStructureMoleculeDriver(
    molecule=molecule,
    basis="sto3g",
    driver_type=ElectronicStructureDriverType.PYSCF,
)

We could now run the driver manually by calling `driver.run()` but the preferred way is to wrap it into the previously mentioned `ElectronicStructureProblem` which we can solve later on.

In [4]:
problem = ElectronicStructureProblem(driver)

Out of curiosity let us inspect the Hamiltonian of our system in its second-quantized form:

In [5]:
# this will call driver.run() internally
second_q_ops = problem.second_q_ops()

In [ ]:
# TODO: move to imports (and show imports)
#FermionicOp.set_truncation(500)

hamiltonian = second_q_ops[0]
print(hamiltonian)

We can gain further insight by investigating the raw output of our driver:

In [7]:
print(problem.grouped_property)

ElectronicStructureDriverResult:
	DriverMetadata:
		Program: PYSCF
		Version: 1.7.6
		Config:
			atom=O 0.0 0.0 0.115;H 0.0 0.754 -0.459;H 0.0 -0.754 -0.459
			unit=Angstrom
			charge=0
			spin=0
			basis=sto3g
			method=rhf
			conv_tol=1e-09
			max_cycle=50
			init_guess=minao
			max_memory=4000
			
	ElectronicBasisTransform:
		Initial basis: atomic
		Final basis: molecular
		Alpha coefficients:
		[0, 0] = 0.9941038468519987
		[0, 1] = -0.2324968442727891
		[0, 2] = 3.07214922262906e-17
		[0, 3] = -0.10305188327114584
		[0, 4] = -4.448901852669351e-18
		[0, 5] = -0.13415815779896753
		[0, 6] = -2.566406001885063e-16
		[1, 0] = 0.02678207865614997
		[1, 1] = 0.8301922072362911
		[1, 2] = -2.667817867739604e-16
		[1, 3] = 0.5369920122052859
		[1, 4] = 5.10370316375746e-17
		[1, 5] = 0.9038656248714041
		[1, 6] = 1.8130095584959163e-15
		[2, 0] = -5.621580737174608e-19
		[2, 1] = -1.353781313386951e-16
		[2, 2] = -7.603134836834508e-17
		[2, 3] = -7.426141965629887e-18
		[2, 4] = 1.0
		[

We notice that this system contains 14 spin orbitals, which will cause our calculations to take a considerably long time. We can reduce the size of our problem by selected an *active space*.

In [8]:
transformer = ActiveSpaceTransformer(
    num_electrons=2,
    num_molecular_orbitals=3,
)

In [9]:
problem_reduced = ElectronicStructureProblem(driver, [transformer])
second_q_ops_reduced = problem_reduced.second_q_ops()
hamiltonian_reduced = second_q_ops_reduced[0]

In [10]:
print(hamiltonian_reduced)

Fermionic Operator
register length=6, number terms=33
  (0.03893103043276141+0j) * ( +_0 -_1 +_3 -_4 )
+ (-0.03893103043276137+0j) * ( +_0 -_1 -_3 +_4 )
+ (0.02435454706021395+0j) * ( +_0 -_2 +_3 -_5 )
+ (-0.024354547060213932+0j) * ( +_0 -_2 -_3 +_5 )
+ (-0.03893103043276137+0j) * ( -_0 +_1 +_3 -_4 )
+ (0.038931030432761346+0j) * ( -_0 +_1 -_3 +_4 )
+ (-0.024354547060213932+0j) * ( -_0 +_2 +_3 -_5 )
+ (0.024354547060213932+0j) * ( -_0 +_2 -_3 +_5 )
+ (0.11590317383979615+0j) * ( +_1 -_2 +_4 -_5 )
+ (-0.11590317383979612+0j) * ( +_1 -_2 -_4 +_5 ...


In [11]:
print(problem_reduced.grouped_property_transformed)

ElectronicStructureDriverResult:
	DriverMetadata:
		Program: PYSCF
		Version: 1.7.6
		Config:
			atom=O 0.0 0.0 0.115;H 0.0 0.754 -0.459;H 0.0 -0.754 -0.459
			unit=Angstrom
			charge=0
			spin=0
			basis=sto3g
			method=rhf
			conv_tol=1e-09
			max_cycle=50
			init_guess=minao
			max_memory=4000
			
	ElectronicBasisTransform:
		Initial basis: atomic
		Final basis: molecular
		Alpha coefficients:
		[0, 0] = 0.9941038468519983
		[0, 1] = -0.2324968442727841
		[0, 2] = 2.05754194645156e-16
		[0, 3] = -0.10305188327114131
		[0, 4] = 1.5795877551576306e-18
		[0, 5] = -0.13415815779898035
		[0, 6] = 3.7565869909763627e-16
		[1, 0] = 0.026782078656150548
		[1, 1] = 0.8301922072362607
		[1, 2] = -8.907197834480505e-16
		[1, 3] = 0.5369920122052277
		[1, 4] = -2.5425597103889975e-17
		[1, 5] = 0.9038656248714654
		[1, 6] = -2.6399582880334174e-15
		[2, 0] = -1.7557510392981973e-19
		[2, 1] = 1.0341540791423741e-16
		[2, 2] = 8.698720334656905e-16
		[2, 3] = -2.5211953822379544e-16
		[2, 4] = 1

## Mapping the Problem to the Qubit Space

In [12]:
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper, ParityMapper

In order to solve our problem with a Quantum algorithm we must map our second-quantized Hamiltonian into a qubit operator. This is done with the `QubitConverter` in combination with a `QubitMapper` of your choice.

The most straight forward mapping is the *Jordan-Wigner* one which stores the *occupation* information of one spin orbital in one qubit:

<img src="jw_mapping.png" width="1200"/>

In [13]:
jw_mapper = JordanWignerMapper()
jw_converter = QubitConverter(jw_mapper)

In [14]:
qubit_op_jw = jw_converter.convert(hamiltonian_reduced)
print(qubit_op_jw)

-0.05135743132368864 * IIIIII
- 0.480940515477597 * ZIIIII
- 0.42585723410847787 * IZIIII
+ 0.11275928505376133 * ZZIIII
- 0.17628591239111302 * IIZIII
+ 0.1505681335893225 * ZIZIII
+ 0.13748636180405338 * IZZIII
- 0.48094051547759703 * IIIZII
+ 0.15537730842446618 * ZIIZII
+ 0.14173507851371037 * IZIZII
+ 0.156656770354376 * IIZZII
- 0.42585723410847776 * IIIIZI
+ 0.14173507851371037 * ZIIIZI
+ 0.14925817605490319 * IZIIZI
+ 0.14721911941224375 * IIZIZI
+ 0.11275928505376133 * IIIZZI
- 0.17628591239111321 * IIIIIZ
+ 0.156656770354376 * ZIIIIZ
+ 0.14721911941224375 * IZIIIZ
+ 0.2200397733437609 * IIZIIZ
+ 0.1505681335893225 * IIIZIZ
+ 0.13748636180405338 * IIIIZZ
+ 0.028975793459949023 * XXIXXI
+ 0.028975793459949023 * YYIXXI
+ 0.028975793459949023 * XXIYYI
+ 0.028975793459949023 * YYIYYI
+ 0.006088636765053485 * XZXXZX
+ 0.006088636765053485 * YZYXZX
+ 0.006088636765053485 * XZXYZY
+ 0.006088636765053485 * YZYYZY
+ 0.009732757608190343 * IXXIXX
+ 0.009732757608190343 * IYYIXX
+ 0.0097

In this step we can reduce the size of our problem further by leveraging symmetries in the Hilbert space of our system. One mapping which supports a straight-forward removal of 2 qubits by exploiting the particle-conserving properties of electronic structure problems, is the *parity* mapping:

In [15]:
parity_mapper = ParityMapper()
parity_converter = QubitConverter(parity_mapper, two_qubit_reduction=True)

In [16]:
qubit_op_parity = parity_converter.convert(hamiltonian_reduced, num_particles=problem_reduced.num_particles)
print(qubit_op_parity)

-0.051357431323688627 * IIII
- 0.6184268772816501 * ZIII
+ (0.2890451974448742+1.3877787807814457e-17j) * IZII
- 0.5764253676978002 * ZZII
+ (0.6184268772816502-2.7755575615628914e-17j) * IIZI
- 0.15537730842446612 * ZIZI
+ 0.15665677035437595 * IZZI
- 0.1417350785137103 * ZZZI
+ (-0.28904519744487445-6.938893903907228e-18j) * IIIZ
+ 0.15665677035437595 * ZIIZ
+ (-0.22003977334376085+1.3877787807814457e-17j) * IZIZ
+ 0.1472191194122437 * ZZIZ
- 0.5764253676978001 * IIZZ
+ 0.1417350785137103 * ZIZZ
- 0.1472191194122437 * IZZZ
+ 0.14925817605490313 * ZZZZ
+ 0.028975793459949013 * XIXI
- 0.028975793459949013 * XZXI
+ (0.028975793459949013+1.734723475976807e-18j) * XIXZ
- 0.028975793459949013 * XZXZ
+ 0.00973275760819034 * IXIX
+ 0.00973275760819034 * ZXIX
- 0.00973275760819034 * IXZX
- 0.00973275760819034 * ZXZX
+ 0.006088636765053483 * XXXX
+ (-0.006088636765053483+4.336808689942018e-19j) * YYXX
- 0.006088636765053482 * XXYY
+ 0.006088636765053483 * YYYY


/home/oss/Files/Qiskit/src/qiskit-terra/main/qiskit/opflow/primitive_ops/pauli_sum_op.py:227: DeprecationWarning: The SparsePauliOp.table method is deprecated as of Qiskit Terra 0.19.0 and will be removed no sooner than 3 months after the releasedate. Use SparsePauliOp.paulis method instead.
  if not np.any(new_self.primitive.table.array):


## Finding the Ground-State Solution

In [17]:
from qiskit.algorithms.optimizers import SLSQP
from qiskit.opflow.expectations import AerPauliExpectation
from qiskit.providers.aer import StatevectorSimulator, QasmSimulator
from qiskit_nature.algorithms.ground_state_solvers import GroundStateEigensolver
from qiskit_nature.algorithms.ground_state_solvers.minimum_eigensolver_factories import VQEUCCFactory

Now that we have converted our problem to a Qubit operator, we need to find it's eigenvalue which corresponds to the ground state of our system.

<img src="H2_gs.png" width="400"/>

Qiskit Aer provides a variety of simulation backends which we can use instead of an actual quantum computer.

One such simulator is the `StatevectorSimulator` which performs an exact simulation of a noiseless quantum computer:

In [23]:
statevector_backend = StatevectorSimulator()

Another simulator which always contains sampling noise and can even simulate qubit and gate infidelities is the `QasmSimulator`:

In [24]:
qasm_backend = QasmSimulator()

In order to use these backends during our `VQE` we can simply use a factory method which constructs the `VQE` instance for us, like so:

In [19]:
vqe_factory = VQEUCCFactory(
    quantum_instance=statevector_backend,
    #quantum_instance=qasm_backend,
    optimizer=SLSQP(),
)

This factory can then be used to construct a `GroundStateEigensolver` with which we can finally solve our problem:

In [20]:
solver = GroundStateEigensolver(parity_converter, vqe_factory)

In [21]:
result = solver.solve(problem_reduced)

In [22]:
print(result)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -84.248285546104
  - computed part:      -1.664149612416
  - ActiveSpaceTransformer extracted energy part: -82.584135933688
~ Nuclear repulsion energy (Hartree): 9.285714221678
> Total ground state energy (Hartree): -74.962571324426
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  0.00377945]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  0.68311967]
    - computed part:      [0.0  0.0  0.43252811]
    - ActiveSpaceTransformer extracted energy part: [0.0  0.0  0.25059156]
  > Dipole moment (a.u.): [0.0  0.0  -0.67934022]  Total: 0.67934022
                 (debye): [0.00000001  0.00000001  -1.72671045]  Total: 1.72671045
 
